# Lossy Counting (Algorithm 3.2)

目的: ストリームをバケットに分割し、各要素に誤差境界 `Δ = ⌊ n/k ⌋` を付与して頻出候補を維持。

処理概要:
- 新規要素 `i` を観測したら `c[i] = 1 + Δ` として登録
- 各アイテム処理後に `Δ` が変わったバケット境界で、 `c[j] < Δ` の要素を一括削除

性質: 誤差は高々 `n/k`。MGより実装が容易で、周期的なクレンジングで高速。


In [ ]:

from collections import defaultdict, Counter
import math

def lossy_counting(stream, k):
    T = {}   # element -> count (with offset)
    n = 0
    Delta = 0 # floor(n/k)
    for i in stream:
        n += 1
        if i in T:
            T[i] += 1
        else:
            T[i] = 1 + Delta
        if (n // k) != Delta:
            Delta = n // k
            # prune
            to_del = [j for j, c in T.items() if c < Delta]
            for j in to_del:
                del T[j]
    return T, n, Delta

# demo
import pandas as pd
df = pd.read_csv("/mnt/data/Brighton v Man City LIVE Watchalong!_chat_log.csv", encoding="utf-8", engine="python")
stream = df["author"].astype(str).tolist()
k = 20
T, n, Delta = lossy_counting(stream, k)

true = Counter(stream)
cands = {u: true[u] for u in T.keys()}
print("n=", n, "Delta=", Delta, "candidates=", len(T))
print(sorted([(u, T[u], cands[u]) for u in T], key=lambda t: -cands[t[0]])[:20])
print("\\nTop-20 ground truth:")
print(true.most_common(20))
